<a href="https://colab.research.google.com/github/Nikhil9490/ML_Algorithms_Revision/blob/main/SVR_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import seaborn as sns

In [2]:
df= sns.load_dataset('tips')

In [3]:
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [5]:
df['sex'].value_counts()

,count
sex,
Male,157
Female,87


In [6]:
df['smoker'].value_counts()

,count
smoker,
No,151
Yes,93


In [7]:
df['day'].value_counts()

,count
day,
Sat,87
Sun,76
Thur,62
Fri,19


In [8]:
df['time'].value_counts()

,count
time,
Dinner,176
Lunch,68


In [9]:
##feature encoding
## label encoding and one hot encoding
# first divide data into independent and dependent features
df.columns


Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [10]:
X= df[['tip', 'sex', 'smoker', 'day', 'time', 'size']]
y= df['total_bill']

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.25, random_state=10)

In [12]:
X_train.head()

,tip,sex,smoker,day,time,size
58,1.76,Male,Yes,Sat,Dinner,2
1,1.66,Male,No,Sun,Dinner,3
2,3.50,Male,No,Sun,Dinner,3
68,2.01,Male,No,Sat,Dinner,2
184,3.00,Male,Yes,Sun,Dinner,2


In [13]:
##feature encoding ( label encoding and one hot encoding)
from sklearn.preprocessing import LabelEncoder
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()

In [14]:
X_train['sex']= le1.fit_transform(X_train['sex'])
X_train['smoker']= le2.fit_transform(X_train['smoker'])
X_train['time']= le3.fit_transform(X_train['time'])

so why do we do fit_transform on training data but only transform on test data?

its to prevent data leakage. model learns female->0, male->1 from traning data and applies on it. thats fit_transform

model applies learned rules on test data. thats transform.

In [15]:
X_train.head()

,tip,sex,smoker,day,time,size
58,1.76,1,1,Sat,0,2
1,1.66,1,0,Sun,0,3
2,3.50,1,0,Sun,0,3
68,2.01,1,0,Sat,0,2
184,3.00,1,1,Sun,0,2


In [16]:
X_test['sex']= le1.transform(X_test['sex'])
X_test['smoker']= le2.transform(X_test['smoker'])
X_test['time']= le3.transform(X_test['time'])

In [17]:
X_test.head()

,tip,sex,smoker,day,time,size
162,2.00,0,0,Sun,0,3
60,3.21,1,1,Sat,0,2
61,2.00,1,1,Sat,0,2
63,3.76,1,1,Sat,0,4
69,2.09,1,1,Sat,0,2


In [18]:
##One hot encoding -- ColumnTransformer

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [19]:
ct = ColumnTransformer(transformers=[('onehot', OneHotEncoder(drop= 'first'),[3])], remainder='passthrough')

#why drop = 'first'?
# sat - 0 sun - 0 thu -0 fri -1
# cant i just assume its friday when remaining are zeroes? so why do we need fri column? so we can remove any one column and assign its value based onn absensce of other possibilities

# passthrough lets all columns be there. if you choose "drop" instead, only changed column will be there. in this case [3]. others will be dropped. passthrough lets other columns be.

In [20]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

X_train = ct.fit_transform(X_train)

In [21]:
X_test = ct.transform(X_test)

In [22]:
X_test

array([[0.  , 1.  , 0.  , 2.  , 0.  , 0.  , 0.  , 3.  ],
       [1.  , 0.  , 0.  , 3.21, 1.  , 1.  , 0.  , 2.  ],
       [1.  , 0.  , 0.  , 2.  , 1.  , 1.  , 0.  , 2.  ],
       [1.  , 0.  , 0.  , 3.76, 1.  , 1.  , 0.  , 4.  ],
       [1.  , 0.  , 0.  , 2.09, 1.  , 1.  , 0.  , 2.  ],
       [0.  , 0.  , 1.  , 5.  , 1.  , 1.  , 1.  , 2.  ],
       [0.  , 1.  , 0.  , 3.51, 1.  , 0.  , 0.  , 2.  ],
       [1.  , 0.  , 0.  , 5.16, 1.  , 1.  , 0.  , 4.  ],
       [0.  , 1.  , 0.  , 5.  , 1.  , 0.  , 0.  , 2.  ],
       [1.  , 0.  , 0.  , 3.6 , 1.  , 0.  , 0.  , 3.  ],
       [0.  , 1.  , 0.  , 5.65, 1.  , 1.  , 0.  , 2.  ],
       [1.  , 0.  , 0.  , 2.5 , 0.  , 1.  , 0.  , 3.  ],
       [0.  , 0.  , 1.  , 1.44, 1.  , 0.  , 1.  , 2.  ],
       [1.  , 0.  , 0.  , 3.09, 0.  , 1.  , 0.  , 4.  ],
       [0.  , 1.  , 0.  , 2.  , 1.  , 0.  , 0.  , 4.  ],
       [0.  , 0.  , 1.  , 1.36, 0.  , 0.  , 1.  , 3.  ],
       [0.  , 0.  , 1.  , 2.  , 0.  , 0.  , 1.  , 2.  ],
       [0.  , 0.  , 1.  , 1.68,

In [23]:
##SVR

from sklearn.svm import SVR
svr = SVR()


In [24]:
svr.fit(X_train, y_train)

SVR()

In [25]:
y_pred = svr.predict(X_test)

In [26]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
print(r2_score(y_test, y_pred))
print(mean_absolute_error(y_test,y_pred))
print(mean_squared_error(y_test,y_pred))

0.46028114561159283
4.1486423210190235
45.692136151683385


In [27]:
#Hyperparameter tuning using GridSearch CV
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.1,1,10,100, 1000], 'gamma': [1,0.1,0.01,0.001, 0.0001], 'kernel': ['rbf']}

In [28]:
grid = GridSearchCV(SVR(), param_grid, refit=True, verbose=3)

grid.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.067 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.058 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.145 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.025 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.089 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.013 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.021 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-0.010 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.124 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.050 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=0.01, kernel=rbf;, score=-0.053 total time=   0.0s
[CV 2/5] END ....C=0.1, gamma=0.01, kernel=rbf;

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [29]:
grid.best_params_

{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}

In [30]:
grid_predictions = grid.predict(X_test)

In [31]:
print(r2_score(y_test, grid_predictions))
print(mean_absolute_error(y_test,grid_predictions))
print(mean_squared_error(y_test,grid_predictions))

0.5081618245078687
3.8685147526100234
41.63859886764171


In [33]:
from sklearn.metrics import accuracy_score
# accuracy_score(y_test, y_pred) # This metric is for classification tasks, not regression.

In [35]:
#adjusted r2
def adjusted_r2(y_true, y_pred, X_data):
    r2 = r2_score(y_true, y_pred)
    n = len(y_true)
    p = X_data.shape[1] # Number of features
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    return adj_r2

# Calculate MAPE (Mean Absolute Percentage Error)
def mean_absolute_percentage_error(y_true, y_pred):
    # Avoid division by zero by adding a small epsilon or handling zero values
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Assuming X_test, y_test, and grid_predictions are available from previous steps
import numpy as np

adj_r2_val = adjusted_r2(y_test, grid_predictions, X_test)
mape_val = mean_absolute_percentage_error(y_test, grid_predictions)

print(f"Adjusted R-squared: {adj_r2_val:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape_val:.4f}%")

Adjusted R-squared: 0.4325
Mean Absolute Percentage Error (MAPE): 18.3753%
